# Is the word count related to the priming?

## based on analysis 22

### 2/11/21

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import pickle
import copy
from random import randrange
from sklearn.metrics import jaccard_score
import random
from sklearn.preprocessing import normalize

from sklearn.decomposition import PCA #for cluster analysis
import matplotlib.pyplot as plt #for plotting
import os #for importing
import pickle #for loading transcripts
from scipy.stats import pearsonr 

%autosave 5

Autosaving every 5 seconds


# 0. Pre
## A. Importing story and template vectors and sums 

In [2]:
templates = pickle.load( open( 'c_template', "rb" ) )
mrecalls = pickle.load( open( 'fr_recalls', "rb" ) )
recalls = pickle.load( open( 'c_recall', "rb" ) )
stories = pickle.load( open( 'c_stories', "rb" ) )
ids = pickle.load( open( 'ids_dict', "rb" ) )
data_dict = pickle.load( open( 'data_dict', "rb" ) )
# sums
sums = pickle.load( open( "new_sums", "rb" ) )

## B. wordCount function

In [3]:
# FastText preprocessing, based on bittlingmayer/ft_wiki_preproc.py
# Remove special characters, put spaces between all tokens
SUB = ["s/’/'/g", "s/′/'/g", "s/''/ /g", "s/'/ ' /g", 's/“/"/g', 's/”/"/g', 's/"/ /g', "s/\\./ \\. /g", "s/<br \\/>/ /g", "s/, / , /g", "s/(/ ( /g", "s/)/ ) /g", "s/\\!/ \\! /g", "s/\\?/ \\? /g", "s/\\;/ /g", "s/\\:/ /g", "s/-/ - /g", "s/=/ /g", "s/=/ /g", "s/*/ /g", "s/|/ /g", "s/«/ /g", 
       "s/…/ /g", "s/‘/ /g", "s/í/ /g", "s/ñ/ /g", "s/\x84/ /g", "s/î/ /g", "s/ó/ /g", "s/\x83/ /g", "s/ï/ /g", "s/õ/ /g",
       "s/ò/ /g", "s/,/ /g", "s/ô/ /g", "s/\x92/ /g", "s/é/ /g", "s/\x8e/ /g", "s/â\x80¦/ /g", "s/\x91/ /g", "s/\x93/ /g",
       "s/\x94/ /g", "s/ã®/ /g", "s/ã¨/ /g", "s/ã©/ /g",
       "s/\â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x9d/ /g", "s/â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x98/ /g",
       "s/â/ /g"]

def __normalize_text(s):
    for sg in SUB:
        rep = sg.replace('\\','').split('/')
        s = s.replace(rep[1], rep[2])
    s = s.replace('/',' ')
    return s

def __spaces(s):
    return ' '.join(s.split())

def __digits(s):
    return ''.join(filter(lambda c: not c.isdigit(), s))

# def preproc(s):
#     return __punctuation(__spaces(__digits(__normalize_text(s.lower()))))

def preproc(s):
    return (__spaces(__digits(__normalize_text(s.lower()))))

def __punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))


def wordCount(sentence, model = 'fasttext'):        
    words = preproc(sentence).split(' ')
    
    return len(words)

# get the word count for all participants in each priming group
### type of output for r: priming, story, wordcount

In [6]:
data = np.zeros((0, 3))

for story in mrecalls:
    for i in range(0, 3):
        if i == 0:
            prime_type = 'no_prime'
        if i == 1:
            prime_type = 'loc'
        if i == 2:
            prime_type = 'soc'
        n_partc = mrecalls[story][i].shape[0]
        for j in range(0, n_partc):
            this_text = mrecalls[story][i][j][0]
            wc = wordCount(this_text)
            new_row = np.array([prime_type, story, wc])
            data = np.vstack((data, new_row))

In [9]:
np.savetxt('priming_wc.csv', data, delimiter=',', fmt='%s', header='priming,story,wordcount', comments='')
